In [4]:
import numpy as np
import pandas as pd
import os
import torch
import tensorflow as tf
import matplotlib.pyplot as plt
import mne
%matplotlib inline


In [5]:
# Set seed for reproducibility
np.random.seed(1000)
tf.random.set_seed(1000)

In [6]:
channels = [
    'EEG FP1-REF',
    'EEG FP2-REF',
    'EEG F3-REF',
    'EEG F4-REF',
    'EEG F7-REF',
    'EEG F8-REF',
    'EEG FZ-REF',
    'EEG A1-REF',
    'EEG A2-REF',
    'EEG T3-REF',
    'EEG T4-REF',
    'EEG C3-REF',
    'EEG C4-REF',
    'EEG CZ-REF',
    'EEG P3-REF',
    'EEG P4-REF',
    'EEG PZ-REF',
    'EEG T5-REF',
    'EEG T6-REF',
    'EEG O1-REF',
    'EEG O2-REF',
]

In [14]:
# Load the dataset
data_path = "train/normal"
os.listdir(data_path)
raw = mne.io.read_raw_edf(os.path.join(data_path, os.listdir(data_path)[0]))
raw = raw.copy().pick_channels(channels)

FileNotFoundError: [Errno 2] No such file or directory: 'train/normal'

In [15]:
raw = raw.copy().resample(100)

NameError: name 'raw' is not defined

In [16]:
raw = raw.copy().crop(tmin=60, tmax=60*7)

NameError: name 'raw' is not defined

In [17]:
raw

NameError: name 'raw' is not defined

In [18]:
raw.plot()

NameError: name 'raw' is not defined

In [19]:
x_train = raw.get_data()

NameError: name 'raw' is not defined

In [20]:
x_train = tf.expand_dims(tf.transpose(x_train), axis = 0)

NameError: name 'x_train' is not defined

In [21]:
x_train.shape

NameError: name 'x_train' is not defined

In [115]:
# Create an SCNet model using tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Dropout, BatchNormalization, AvgPool1D, ReLU, Softmax
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAvgPool1D, Concatenate, SpatialDropout1D, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

class GSPLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(GSPLayer, self).__init__()
    
    def call(self, x, gap):
        return tf.expand_dims(tf.sqrt(tf.reduce_sum((x - gap) ** 2, axis=1)) / 21, 1)

def mffm_block(input):
    # Convolutional layer with ReLU activation
    x1 = Conv1D(filters=8, kernel_size=5, padding='same', data_format='channels_first')(input)
    x1 = BatchNormalization(axis=1)(x1)
    x1 = ReLU()(x1)

    # Concatenate input and x1 along the channel axis
    x2 = Concatenate(axis=1)([input, x1])
    x2 = Conv1D(filters=16, kernel_size=5, padding='same', data_format='channels_first')(x2)
    x2 = BatchNormalization(axis=1)(x2)
    x2 = ReLU()(x2)

    # Concatenate input, x1, and x2 along the channel axis
    x3 = Concatenate(axis=1)([input, x1, x2])
    return x3


def scnet(input_shape):
    input = Input(shape=input_shape)
    # Implementing SILM
    gap = GlobalAvgPool1D(data_format='channels_last',keepdims=True)(input)
    gmp = GlobalMaxPool1D(data_format='channels_last',keepdims=True)(input)
    gsp = GSPLayer()(input, gap)
    # Apply dropout on each
    gap = Dropout(0.05)(gap)
    gmp = Dropout(0.05)(gmp)
    gsp = Dropout(0.05)(gsp)
    # Concatenate with input
    x = Concatenate(axis=1)([input, gap, gmp, gsp])

    # Take avg and maxpool
    x1 = MaxPool1D(3, data_format='channels_first', strides = 3)(x)
    x2 = AvgPool1D(3, data_format='channels_first', strides = 3)(x)
    x = Concatenate(axis=2)([x1, x2])
    x = BatchNormalization(axis = 1)(x)
    
    x1 = mffm_block(x)
    x2 = mffm_block(x)
    x = Add()([x1, x2])
    
    # Apply spatial dropout
    x = SpatialDropout1D(0.5)(x)
    x = MaxPool1D(2, strides = 2, data_format='channels_first')(x)
    x = Conv1D(32, 3, data_format='channels_first')(x)
    x = BatchNormalization(axis=1)(x)
    x = ReLU()(x)

    x1 = mffm_block(x)
    x2 = mffm_block(x)
    x = Add()([x1, x2])

    x = Conv1D(32, 3, data_format='channels_first')(x)
    x = BatchNormalization(axis=1)(x)
    x = mffm_block(x)
    x = MaxPool1D(2, strides = 2, data_format='channels_first')(x)
    x = Conv1D(32, 3, data_format='channels_first')(x)
    x = BatchNormalization(axis=1)(x)

    # Apply gap
    x = GlobalAvgPool1D(data_format='channels_last',keepdims=True)(x)

    # Flatten the output
    x = Flatten()(x)
    x = Dense(2)(x)

    # Apply softmax
    outputs = Softmax()(x)
    return Model(inputs=input, outputs=outputs)

In [116]:
model = scnet((21, 42001))

model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [117]:
model.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_46 (InputLayer)       [(None, 21, 42001)]          0         []                            
                                                                                                  
 global_average_pooling1d_6  (None, 1, 42001)             0         ['input_46[0][0]']            
 2 (GlobalAveragePooling1D)                                                                       
                                                                                                  
 global_max_pooling1d_45 (G  (None, 1, 42001)             0         ['input_46[0][0]']            
 lobalMaxPooling1D)                                                                               
                                                                                           

In [24]:
y_train = np.array([0])
y_train = tf.one_hot(y_train, 2)

In [25]:
model.fit(x_train, y_train, epochs=10)

NameError: name 'x_train' is not defined